## Heuristic Comparison: Basic vs Constraint-Aware A*
This notebook demonstrates the difference between basic Euclidean A* and a constraint-aware A* where edge penalties (e.g., stairs, slope, etc.) impact pathfinding decisions.

In [ ]:

import networkx as nx
import heapq
import matplotlib.pyplot as plt

# Define points (coordinates)
points = {
    'A': (-37.8156505, 145.2294636),
    'B': (-37.8154740, 145.2294588),
    'C': (-37.8152886, 145.2288074),
    'D': (-37.8148201, 145.2290135),
    'E': (-37.8149849, 145.2290695),
    'F': (-37.8140786, 145.2296805),
    'G': (-37.8144825, 145.2296467),
    'H': (-37.8139861, 145.2306309),
    'I': (-37.8144433, 145.2300744),
    'J': (-37.8136444, 145.2314192),
    'K': (-37.8146830, 145.2292295),
    'L': (-37.8140991, 145.2303492),
    'M': (-37.8148709, 145.2295034)
}

segments = [
    ('A', 'B', 20), ('B', 'C', 65), ('C', 'D', 55), ('D', 'E', 19), ('E', 'F', 80), 
    ('F', 'G', 45), ('G', 'H', 95), ('H', 'I', 70), ('I', 'J', 110), ('J', 'K', 150),
    ('K', 'L', 90), ('L', 'M', 65), ('M', 'A', 128), ('D', 'K', 43), ('C', 'G', 43),
    ('E', 'G', 57), ('G', 'I', 61), ('F', 'I', 88), ('K', 'I', 78), ('H', 'J', 143),
    ('B', 'E', 85), ('A', 'D', 110), ('B', 'F', 130), ('D', 'F', 90), ('H', 'K', 140),
    ('C', 'F', 100), ('M', 'J', 160), ('L', 'H', 120), ('M', 'I', 85), ('E', 'J', 180)
]

constraints = {
    ('C', 'D'): 'stairs', ('E', 'F'): 'kerb_ramp', ('H', 'I'): 'lift',
    ('C', 'F'): 'obstacle', ('J', 'K'): 'slope', ('K', 'I'): 'slope',
    ('A', 'D'): 'stairs', ('L', 'H'): 'lift', ('M', 'J'): 'slope', ('B', 'F'): 'kerb_ramp'
}

# Build graph
G = nx.Graph()
for point, (lat, lon) in points.items():
    G.add_node(point, pos=(lon, lat))
for u, v, dist in segments:
    G.add_edge(u, v, cost=dist)


In [ ]:

def heuristic_basic(node, goal, points):
    x1, y1 = points[node]
    x2, y2 = points[goal]
    return ((x2 - x1)**2 + (y2 - y1)**2) ** 0.5 * 1000


In [ ]:

def a_star_basic(graph, start, goal, points):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {node: float('inf') for node in graph.nodes}
    f_score = {node: float('inf') for node in graph.nodes}
    g_score[start] = 0
    f_score[start] = heuristic_basic(start, goal, points)

    while open_set:
        current = heapq.heappop(open_set)[1]
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1], g_score[goal]

        for neighbor in graph.neighbors(current):
            tentative_g = g_score[current] + graph[current][neighbor]['cost']
            if tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic_basic(neighbor, goal, points)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None, float('inf')


In [ ]:

def a_star_constraints(graph, start, goal, points, constraints):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {node: float('inf') for node in graph.nodes}
    f_score = {node: float('inf') for node in graph.nodes}
    g_score[start] = 0
    f_score[start] = heuristic_basic(start, goal, points)

    while open_set:
        current = heapq.heappop(open_set)[1]
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1], g_score[goal]

        for neighbor in graph.neighbors(current):
            base_cost = graph[current][neighbor]['cost']
            edge = (current, neighbor) if (current, neighbor) in constraints else (neighbor, current)
            terrain = constraints.get(edge, None)

            penalty = {
                'stairs': 100,
                'slope': 40,
                'lift': 20,
                'kerb_ramp': 10,
                'obstacle': 9999
            }.get(terrain, 0)

            tentative_g = g_score[current] + base_cost + penalty
            if tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic_basic(neighbor, goal, points)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None, float('inf')


In [ ]:

# Test cases
test_cases = [('A', 'F'), ('C', 'H'), ('E', 'J'), ('K', 'M'), ('B', 'I')]

for start, goal in test_cases:
    path_basic, cost_basic = a_star_basic(G, start, goal, points)
    path_constr, cost_constr = a_star_constraints(G, start, goal, points, constraints)
    print(f"From {start} to {goal}")
    print(f"  Basic A*:      {path_basic} | Cost: {int(cost_basic)}m")
    print(f"  Constrained A*: {path_constr} | Cost: {int(cost_constr)}m")
    print("---")

    #vector<double> vector2= new vector <double>()
#vector2 insert (4,5)
